## Predict Amazon Product Prices

### An example fine-tuned model using Llama3.1

### Resuming fine-tuning from a checkpoint saved to the HuggingFace Hub



In [ ]:
# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece wandb matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# imports
# With much thanks to Islam S. for identifying that there was a missing import!

import os
import re
import math
from tqdm import tqdm
# from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Constants
# pricer-2025-04-20_15.45.11
# a955e367cb687dba8d4958524ad8c361e3f297a6

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "lubhu"
DATASET_NAME = f"{HF_USER}/pricer-data"
ORIGINAL_FINETUNED = f"{HF_USER}/{PROJECT_NAME}-2025-04-20_15.45.11"
ORIGINAL_REVISION = "a955e367cb687dba8d4958524ad8c361e3f297a6" # the commit hash in the HuggingFace repo
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"


# Hyperparameters for QLoRA Fine-Tuning

EPOCHS = 1
LORA_ALPHA = 32
LORA_R = 16
LORA_DROPOUT = 0.05
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-5 # 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WEIGHT_DECAY = 0.001
WARMUP_RATIO = 0.05
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
MAX_SEQUENCE_LENGTH = 182
QUANT_4_BIT = True

# Other config

STEPS = 50
SAVE_STEPS = 1000
LOG_TO_WANDB = True

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"

%matplotlib inline

### Log in to HuggingFace and Weights & Biases

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

Repeat this for weightsandbiases at https://wandb.ai and add a secret called `WANDB_API_KEY`

In [3]:
# Log in to HuggingFace
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

# # Log in to HuggingFace

# hf_token = userdata.get('HF_TOKEN')
# login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Log in to Weights & Biases
wandb_api_key = os.getenv('WANDB_API_KEY', 'your-key-if-not-using-env')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lubhawan-meena (lubhawan-meena-indian-institute-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [6]:
train[0]['text']

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

In [7]:
if LOG_TO_WANDB:
  details = {
      "base_model": BASE_MODEL,
      "dataset": DATASET_NAME,
      "size": len(train),
      "epochs": EPOCHS,
      "alpha": LORA_ALPHA,
      "r": LORA_R,
      "dropout": LORA_DROPOUT,
      "batch_size": BATCH_SIZE,
      "gradient_accumulation": GRADIENT_ACCUMULATION_STEPS,
      "lr": LEARNING_RATE,
      "lr_scheduler": LR_SCHEDULER_TYPE,
      "max_sequence_length": MAX_SEQUENCE_LENGTH,
      "quant_4_bit": QUANT_4_BIT
  }

  wandb.init(
      project=PROJECT_NAME,
      name=RUN_NAME,
      config=details
  )

## Now load the Tokenizer and Model

The model is "quantized" - we are reducing the precision to 4 bits.

In [8]:
# pick the right quantization

if QUANT_4_BIT:
  print("Loaded in 4 bit")
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

Loaded in 4 bit


In [9]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
    use_cache = False
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

# Load the fine-tuned model with PEFT
if ORIGINAL_REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, ORIGINAL_FINETUNED, revision=ORIGINAL_REVISION, is_trainable=True)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, ORIGINAL_FINETUNED, is_trainable=True)

fine_tuned_model.train()

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 5591.5 MB
Memory footprint: 5646.1 MB


## Now resume Fine-Tuning



In [10]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# First, specify the configuration parameters for LoRA

peft_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_params = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim="paged_adamw_32bit",
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True,
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters
# (Thank you Zoya M for noticing that this needed to be updated for new HuggingFace code)

fine_tuning = SFTTrainer(
    model=fine_tuned_model,
    train_dataset=train,
    peft_config=peft_parameters,
    args=train_params,
    data_collator=collator,
)

# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,1.840200
100,1.830200
150,1.829100
200,1.812500
250,1.816200
300,1.807700
350,1.819500
400,1.843200
450,1.829500
500,1.834200


wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-1000)... Done. 0.3s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-2000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-3000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-4000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-5000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-6000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-7000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-8000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-9000)... Done. 0.2s
wandb: Adding directory to artifact (.\pricer-2025-05-20_09.20.33\checkpoint-10000)... Done. 0.2s
wandb: Adding directory to ar

In [ ]:
if LOG_TO_WANDB:
  wandb.finish()

train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
train/grad_norm,▅▂▄▄▃▅▃▃▃▂▄▂▃▆▂▃▄▁▄▃▃▅▆▆▄▅▆▄▅█▄▄▆▆▅▆█▄▅▇
train/learning_rate,▃▆█████████▇▇▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▂▂▂▂▂▂▂▁▁▁
train/loss,▇▆▆▇▆██▇▇▇▆▅▇▆▆▆▆▅▇▆▅▃▃▃▃▃▂▃▃▃▂▃▁▁▂▂▁▂▁▂
total_flos,6.427628122376503e+18
train/epoch,2
train/global_step,50000
train/grad_norm,11.5198
train/learning_rate,0
train/loss,1.21
